# Ollama example

In [ ]:
# install package
%pip install -U langchain-ollama

In [ ]:
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [ ]:
!ollama pull llama3.2

# Answers and questions

In [ ]:
model = OllamaLLM(model="llama3.2")

In [ ]:
print(model.invoke("what si langchain?"))

In [ ]:
print(model.invoke("tell me a joke"))

## Defining steps

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

response = chain.invoke({"question": "What is LangChain?"})
print(response)

## Agent workflow

In [ ]:
model = OllamaLLM(model="llama3.2")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI customer support for a sports items store. Your name is {name}." ),
    ("human", "{user_input}"),
])

prompt_value = {
    "name": "Bob",
    "user_input": "What is your name and purpose?"
    }

In [ ]:

chain = template | model

In [ ]:
print(chain.invoke(prompt_value))

In [ ]:
prompt_value = {
    "name": "Bob",
    "user_input": "What items do you have for football?"
    }

In [ ]:
print(chain.invoke(prompt_value))

## YELP example

In [ ]:

model = OllamaLLM(model="llama3.2")
template = ChatPromptTemplate([
    ("system", "You are an AI agent the analyses the sentiment of sentences, if it's positive or negative, the vaue ranges should be in between -1 and 1, -1 negative, 0 neutral and 1 positive"),
    ("human", "{user_input}"),
])

prompt_value = {
    "user_input": "What is your name and purpose?"
    }

chain = template | model

In [ ]:
url = 'https://github.com/berinde/curs-analiza-datelor-complexe/blob/main/data/input/3.input_data_prepped_bow.csv?raw=True'
reviews = pd.read_csv(url)
reviews['text'] = reviews['text'].astype(str)
reviews['text_prep'] = reviews['text_prep'].astype(str)
reviews['text_prep_lim'] = reviews['text_prep_lim'].astype(str)
reviews.head(2)

In [ ]:
sentece = reviews['text_prep'][0]

In [ ]:
print(chain.invoke({"user_input":sentece}))

In [ ]:
model = OllamaLLM(model="llama3.2")
template = ChatPromptTemplate([
    ("system", "You are an AI agent the analyses the sentiment of sentences, if it's positive or negative, the vaue ranges should be in between -1 and 1, -1 negative, 0 neutral and 1 positive, you should only provide the score"),
    ("human", "{user_input}"),
])

prompt_value = {
    "user_input": "What is your name and purpose?"
    }

chain = template | model

In [ ]:
print(chain.invoke({"user_input":sentece}))

In [ ]:
reviews['afinn_score_tp'] = reviews['text_prep'][0:5].apply(lambda x: chain.invoke({"user_input":x}))

In [ ]:
reviews['afinn_score_tp']

# Anthropic agent example

https://python.langchain.com/docs/tutorials/agents/

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install -U langchain-community tavily-python
!pip install -U langchain-anthropic

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"]=""

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

## Create the agent

https://python.langchain.com/docs/tutorials/agents/

In [ ]:
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()